# Harry Potter Sentiment-Analyse

In dieser Arbeit werden die Originaltexte von Harry Potter sowie verschiedener
Fan-Fiction-Texte behandelt.

Mittels einer Netzwerkanalyse werden die Beziehungen zwischen den handelnden 
Personen untersucht.
Dabei wird die Python-Bibliothek SpaCy verwendet, um die *Named Entity
Recognition* durchzuführen.

Anschließend werden die gefundenen Namen mit Variationen davon gematcht.
Beispielsweise werden "Harry" und "Harry Potter" nicht als eine Entity erkannt.

Darauffolgend sollen die Interaktionen zwischen handelnden Personen erkannt 
werden.
Dabei wird untersucht, ob ein Satz mehrere Figuren erwähnt.
Mittels des BERT-Modells wird eine Sentimentanalyse durchgeführt, um das 
Sentiment der einzelnen Sätze zu untersuchen.

Gibt es in dem Satz zwei handelnde Personen, dann wird der zugehörige 
Sentiment-Score auf die jeweilige Beziehung zwischen zwei Figuren addiert.

Der ermittelte Graph wird visuell dargestellt.
Dabei entsprechen die Knoten einzelnen Figuren, die Kanten den Beziehungen der 
Figuren untereinander.

Knoten werden je nachdem, ob sie positiv oder negativ beschrieben werden,
eher grün (positiv) oder rot (negativ) dargestellt.
Die Kanten werden dicker, je mehr Interaktionen es zwischen den Figuren gibt.
Auch diese werden je nachdem, ob es sich um positive oder negative Interaktionen
handelt, eher grün (positiv) oder eher rot (negativ) eingefärbt.


In [1]:

'''
Eine Figur.
- Name, unter der die Person referenziert wird
- Aliase, unter der eine Person auch bekannt ist
- einer Liste von Sätzen, in denen die Person alleine vorkommt
- ein Sentiment Value, der aus dem Sentiment der einzelnen Sätze ermittelt wird.
'''
class Figure:

	name = ""
	aliases = []
	sentences = []
	sentiment_value = 0

	def __init__(self, name: str) -> None:
		self.name = name
		self.aliases = [name]

	def add_name(self, name) -> None:
		self.aliases.append(name)

In [2]:
from typing import List

'''
Eine Beziehung zwischen zwei oder mehr Figuren.
- Satz, in dem zwei oder mehr Personen erwähnt werden
- Sentiment Score, der aus dem Satz berechnet wird
- Figuren, die in dem Satz erwähnt werden
'''
class Relation:

	sentence = None
	sentiment_score = 0
	figures = []

	def __init__(self, sentence, sentiment_score, figures: List[Figure]) -> None:
		self.sentence = sentence
		self.sentiment_score = sentiment_score
		self.figures = figures

In [3]:
from curses import nl
from typing import List

'''
Eine Story, die handelnde Personen und Beziehungen zusammenfasst.
- Name der Story
- Figuren der Story
- Beziehungen zwischen den Figuren
- Ein SpaCy-Objekt, das den gesamten Text vorhält
'''
class Story:
	name = ""
	figures = []
	relations = []
	nlp = None

	def __init__(self, name, nlp) -> None:
		self.name = name
		self.nlp = nlp


	def find_figure_by_name(self, name):
		for f in self.figures:
			if name in f.aliases:
				return f
		return None

## Laden der Bücher oder Fan-Fictions

Im Folgenden können die Texte geladen werden, auf denen die Analyse ausgeführt wird.
Dazu müssen sich die Dateien im Unterordner `data` befinden und wie folgt heißen:

|Buch|Dateiname|
|----|---------|
|Harry Potter und der Stein der Weisen|Harry Potter und der Stein der Weisen (German Edition).txt|
|Harry Potter und die Kammer des Schreckens|Harry Potter und die Kammer des Schreckens (German Edition).txt|
|Harry Potter und der Gefangene von Askaban|Harry Potter und der Gefangene von Askaban (German Edition).txt|
|Harry Potter und der Feuerkelch|Harry Potter und der Feuerkelch (German Edition).txt|
|Harry Potter und der Orden des Phönix|Harry Potter und der Orden des Phnix  5 (German Edition).txt|
|Harry Potter und der Halbblutprinz|Harry Potter und der Halbblutprinz (German Edition).txt|
|Harry Potter und die Heiligtümer des Todes|Harry Potter und die Heiligtmer des Todes (German Edition).txt|
|Harry Potter und das verwunschene Kind|Harry Potter und das verwunschene Kind. Teil eins und zwei (Bhnenfassung) (German Edition).txt|
|Quidditch im Wandel der Zeiten|Quidditch im Wandel der Zeiten (Hogwarts Schulbcher) (German Edition).txt|
|Die Marchen von Beedle dem Barden|Die Marchen von Beedle dem Barden (Hogwarts Schulbcher) (German Edition).txt|
|Hogwarts - Ein unvollständiger und unzuverlässiger Leitfaden|Hogwarts  Ein unvollstndiger und unzuverlssiger Leitfaden (Kindle Single) (Pottermore Presents) (German Edition).txt|
|Kurzgeschichten aus Hogwarts  Heldentum, Hrteflle und hanebchene Hobbys|Kurzgeschichten aus Hogwarts  Heldentum, Hrteflle und hanebchene Hobbys (Kindle Single) (Pottermore Presents) (German Edition).txt|
|Kurzgeschichten aus Hogwarts  Macht, Politik und nervttende Poltergeister|Kurzgeschichten aus Hogwarts  Macht, Politik und nervttende Poltergeister (Kindle Single) (Pottermore Presents) (German Edition).txt|
|Phantastische Tierwesen und wo sie zu finden sind  Das Originaldrehbuch|Phantastische Tierwesen und wo sie zu finden sind  Das Originaldrehbuch (German Edition).txt|
|Phantastische Tierwesen und wo sie zu finden sind|Phantastische Tierwesen und wo sie zu finden sind (Hogwarts Schulbcher) (German Edition).txt|

|Fan-Fiction|Dateiname|
|----|---------|
|FF1|ff1.txt|

In [4]:
import spacy

def load_story(name, filename) -> Story:
	sp = spacy.load("de_dep_news_trf")
	nlp = None
	with open("./data/" + filename, "r") as f:
		nlp = sp(f.read())
	return Story(name, nlp)

In [5]:
# Laden der Bücher

story = load_story("Harry Potter und der Stein der Weisen", "Harry Potter und der Stein der Weisen (German Edition).txt")
# story = load_story("Harry Potter und die Kammer des Schreckens", "Harry Potter und die Kammer des Schreckens (German Edition).txt")
# story = load_story("Harry Potter und der Gefangene von Askaban", "Harry Potter und der Gefangene von Askaban (German Edition).txt")
# story = load_story("Harry Potter und der Feuerkelch", "Harry Potter und der Feuerkelch (German Edition).txt")
# story = load_story("Harry Potter und der Orden des Phönix", "Harry Potter und der Orden des Phnix  5 (German Edition).txt")
# story = load_story("Harry Potter und der Halbblutprinz", "Harry Potter und der Halbblutprinz (German Edition).txt")
# story = load_story("Harry Potter und die Heiligtümer des Todes", "Harry Potter und die Heiligtmer des Todes (German Edition).txt")
# story = load_story("Harry Potter und das verwunschene Kind", "Harry Potter und das verwunschene Kind. Teil eins und zwei (Bhnenfassung) (German Edition).txt")
# story = load_story("Quidditch im Wandel der Zeiten", "Quidditch im Wandel der Zeiten (Hogwarts Schulbcher) (German Edition).txt")
# story = load_story("Die Marchen von Beedle dem Barden", "Die Marchen von Beedle dem Barden (Hogwarts Schulbcher) (German Edition).txt")
# story = load_story("Hogwarts - Ein unvollständiger und unzuverlässiger Leitfaden", "Hogwarts  Ein unvollstndiger und unzuverlssiger Leitfaden (Kindle Single) (Pottermore Presents) (German Edition).txt")
# story = load_story("Kurzgeschichten aus Hogwarts  Heldentum, Hrteflle und hanebchene Hobbys", "Kurzgeschichten aus Hogwarts  Heldentum, Hrteflle und hanebchene Hobbys (Kindle Single) (Pottermore Presents) (German Edition).txt")
# story = load_story("Kurzgeschichten aus Hogwarts  Macht, Politik und nervttende Poltergeister", "Kurzgeschichten aus Hogwarts  Macht, Politik und nervttende Poltergeister (Kindle Single) (Pottermore Presents) (German Edition).txt")
# story = load_story("Phantastische Tierwesen und wo sie zu finden sind  Das Originaldrehbuch", "Phantastische Tierwesen und wo sie zu finden sind  Das Originaldrehbuch (German Edition).txt")
# story = load_story("Phantastische Tierwesen und wo sie zu finden sind", "Phantastische Tierwesen und wo sie zu finden sind (Hogwarts Schulbcher) (German Edition).txt")

# TODO: Fan-Fictions raussuchen

/Users/daniel/projects/harry-potter-sentiment/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [20]:
doc = story.nlp
sp = spacy.load("de_dep_news_trf")
for s in doc.sents:
	print(str(s).strip())

AttributeError: 'spacy.tokens.span.Span' object has no attribute 'strip'